In [3]:
%pip install boto3


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# Generate Terms Related to Query
question = "Identify all British Columbia regulations that contain clauses that specify how the regulation applies to goods or services originating outside the province"

In [1]:
# Get nodes with text that match the above terms.
from JEDI_Search_Task.Neo4jRetrieval import Neo4jRetrieval

NEO4J_URI = "bolt://" + "neo4j:7687" 
NEO4J_USERNAME = "admin"  
NEO4J_PASSWORD = "admin" 

# TODO: Remove this and replace with terms from previous step
terms = ["trade", "provincial law", "commerce", "regulation", "market"]

neo4j_worker = Neo4jRetrieval(NEO4J_URI, NEO4J_USERNAME, NEO4J_PASSWORD)
nodes = neo4j_worker.search_many(terms)
print(len(nodes), "nodes found")
neo4j_worker.close()

14055 nodes found


In [5]:
import os
from JEDI_Search_Task.BedrockQuery import BedrockQuery

# Replace this with updated keys if necessary
AWS_ACCESS_KEY_ID = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET_ACCESS_KEY =  os.getenv("AWS_SECRET_ACCESS_KEY")
bedrock = BedrockQuery(AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY)

In [2]:
import json
from concurrent.futures import ThreadPoolExecutor


def create_prompt(question, node):
  return f"""
    Your task is to determine if the following object is related to the question: "{question}".

    Object: {node}

    If the object is related, "1". If it is not related, "0".
    Do not return any explanation, just the number.

    For example:
    Question: "What is the capital of France?"
    Object: {{ "text": "Paris is the capital of France." }}
    Response: 1
    """

# Filter out nodes that are not related to the query using LLM.
# For each node, decide if it is related to the query.
related_nodes = []

def add_or_dispose(node):
     # Use an LLM to determine if the node is relevant.
    prompt = create_prompt(question, node)
    bedrock_response = bedrock.get_response(prompt)
    try:
        if int(bedrock_response.strip()):
            related_nodes.append(node)
            print("good",node.get("text"))
        else:
            print("bad", node.get("text"))
    except:
        pass
    
with ThreadPoolExecutor(4) as executor:
  executor.map(add_or_dispose, nodes)
   
print(len(related_nodes), "related nodes found")
# Save the related nodes to a file.
with open("related_nodes.jsonl", "w") as f:
    for node in related_nodes:
        f.write(json.dumps(node) + "\n")

14055 related nodes found


In [6]:
# Generate report with the remaining information.
# Load nodes from file
with open("related_nodes.jsonl", "r") as f:
    related_nodes = [json.loads(line) for line in f]
    related_nodes = related_nodes[:100]  # Limit to the first 100 nodes for brevity TODO: Remove limit
    formatted_nodes = [{"document": node.get("regId") or node.get("actId"), "section_number": node.get("sectionNumber")} for node in related_nodes]
    # Create a report with the remaining information.
    prompt = f"""
    You are a report generator. You take a list of document titles and section numbers and generate a table to outline them all.

    Given this list of nodes, generate a table that outlines the documents and sections found.
    The table should have two columns: Document and Section Number.
    The Document column should contain the document title, and the Section Number column should contain the section number.
    Do not repeat the same combination of document and section number.
    The table should be formatted in Markdown.
    Here is the list of nodes:
    {json.dumps(formatted_nodes, indent=2)}
    """
    bedrock_response = bedrock.get_response(prompt)
    print("Report generated:")
    print(bedrock_response.strip())


Report generated:
Here's the table outlining the documents and sections found, formatted in Markdown:

| Document | Section Number |
|----------|----------------|
| Clean Energy Act | 2 |
| Clean Energy Act | 28 |
| Clean Energy Act | 29 |
| Coastal Ferry Act | 5 |
| Coastal Ferry Act | 6 |
| Coastal Ferry Act | 7 |
| Coastal Ferry Act | 11 |
| Coastal Ferry Act | 26 |
| Adult Guardianship Act | 46 |
| An Act to Incorporate the Greater Vancouver Sewerage and Drainage District | 7B |
| Animal Health Act | 1 |
| Animal Health Act | 59 |
| Animal Health Act | 93 |
| BC-Alcan Northern Development Fund Act | 7 |
| British Columbia Railway Act | 6 |
| British Columbia Transit Act | 6 |
| Business Corporations Act | 1 |
| Business Corporations Act | 184 |
| Business Practices and Consumer Protection Act | 19 |
| Business Practices and Consumer Protection Act | 21 |
| Business Practices and Consumer Protection Act | 142.1 |
| Business Practices and Consumer Protection Act | 209 |
| College and